In [0]:
cd /content/drive/My Drive/ITSP-2020

In [0]:
pip show tensorflow

Name: tensorflow
Version: 1.10.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: opensource@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: astor, numpy, tensorboard, absl-py, wheel, gast, termcolor, setuptools, grpcio, protobuf, six
Required-by: fancyimpute


In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import random
import json
import os
import time


import tensorflow as tf

import tensorflow.contrib.legacy_seq2seq as seq2seq

from sklearn.model_selection import train_test_split

In [0]:
xo = np.load("iseq_n.npy")
yo = np.load("oseq_n.npy")

In [0]:
xo.shape

(16879, 35, 35)

In [0]:
x = xo[:,:26,:]
y = yo[:,:26]

In [0]:
nxchars = x.shape[2]
print(nxchars)
ltokens = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '-', '+', '=', '#leq', '#neq', '#geq', '#alpha',
                            '#beta', '#lambda', '#lt', '#gt', 'x', 'y', '^', '#frac', '{', '}' ,' ','#int','d','#sinx','#cosx','#tanx']
print("#ltokens: ", len(ltokens))

35
#ltokens:  33


In [0]:
x_seq_length = len(x[0])
y_seq_length = len(y[0])- 1
print(x_seq_length, y_seq_length)

26 25


In [0]:
def batch_data(x, y, batch_size):
    shuffle = np.random.permutation(len(x))
    start = 0
#     from IPython.core.debugger import Tracer; Tracer()()
    x = x[shuffle]
    y = y[shuffle]
    while start + batch_size <= len(x):
        yield x[start:start+batch_size], y[start:start+batch_size]
        start += batch_size

In [0]:
tf.reset_default_graph()

In [0]:
batch_size = 512
nodes = 256
embed_size = 20

# Tensor where we will feed the data into graph
inputs = tf.placeholder(tf.float32, (None, x_seq_length, nxchars), 'inputs')
outputs = tf.placeholder(tf.int32, (None, None), 'output')
targets = tf.placeholder(tf.int32, (None, None), 'targets')

# Embedding layers
output_embedding = tf.Variable(tf.random_uniform((len(ltokens)+1, embed_size), -1.0, 1.0), name='dec_embedding')
date_output_embed = tf.nn.embedding_lookup(output_embedding, outputs)

with tf.variable_scope("encoding") as encoding_scope:
    lstm_enc = tf.contrib.rnn.BasicLSTMCell(nodes)
    _, last_state = tf.nn.dynamic_rnn(lstm_enc, inputs=inputs, dtype=tf.float32)

with tf.variable_scope("decoding") as decoding_scope:
    lstm_dec = tf.contrib.rnn.BasicLSTMCell(nodes)
    dec_outputs, _ = tf.nn.dynamic_rnn(lstm_dec, inputs=date_output_embed, initial_state=last_state)
#connect outputs to 
logits = tf.contrib.layers.fully_connected(dec_outputs, num_outputs=len(ltokens)+1, activation_fn=None) 
with tf.name_scope("optimization"):
    # Loss function
    loss = tf.contrib.seq2seq.sequence_loss(logits, targets, tf.ones([batch_size, y_seq_length]))
    # Optimizer
    optimizer = tf.train.RMSPropOptimizer(1e-3).minimize(loss)

Train the graph above:

In [0]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
print(y_train[0])

[33 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32
 32 32]


In [0]:
def save(sess):
    saver = tf.train.Saver(None)

    save_path = saver.save(sess, save_path="seq_mod/model", global_step=None)
    print('model saved at %s' % save_path)

In [0]:
def restore(sess):
    saver = tf.train.Saver(None)
    path = "seq_mod/model"
    saver.restore(sess, save_path=path)
    print('model restored from %s' % path)

In [0]:
with tf.Session() as sess: 
    sess.run(tf.global_variables_initializer())
    restore(sess)   ####remove this line if training for the first time !!!
    epochs = 200
    for epoch_i in range(epochs):
        start_time = time.time()
        for batch_i, (source_batch, target_batch) in enumerate(batch_data(X_train, y_train, batch_size)):
            _, batch_loss, batch_logits = sess.run([optimizer, loss, logits],
                feed_dict = {inputs: source_batch,
                 outputs: target_batch[:, :-1],
                 targets: target_batch[:, 1:]})
            

        accuracy = np.mean(batch_logits.argmax(axis=-1) == target_batch[:,1:])
        print('Epoch {:3} Loss: {:>6.3f} Accuracy: {:>6.4f} Epoch duration: {:>6.3f}s'.format(epoch_i, batch_loss, 
                                                                          accuracy, time.time() - start_time))

        source_batch, target_batch = next(batch_data(X_test, y_test, batch_size))

        dec_input = np.zeros((len(source_batch), 1)) + len(ltokens)
        for i in range(y_seq_length):
            batch_logits = sess.run(logits,
                        feed_dict = {inputs: source_batch,
                         outputs: dec_input})
            prediction = batch_logits[:,-1].argmax(axis=-1)
            dec_input = np.hstack([dec_input, prediction[:,None]])

        print('Accuracy on test set is: {:>6.3f}'.format(np.mean(dec_input == target_batch)))     
        if epoch_i % 5 == 0:
            save(sess)
    
    save(sess)

INFO:tensorflow:Restoring parameters from seq_mod/model
model restored from seq_mod/model
Epoch   0 Loss:  0.223 Accuracy: 0.9287 Epoch duration: 45.191s
Accuracy on test set is:  0.698
model saved at seq_mod/model
Epoch   1 Loss:  0.229 Accuracy: 0.9263 Epoch duration: 51.391s
Accuracy on test set is:  0.679
Epoch   2 Loss:  0.239 Accuracy: 0.9234 Epoch duration: 44.950s
Accuracy on test set is:  0.681
Epoch   3 Loss:  0.252 Accuracy: 0.9176 Epoch duration: 45.049s
Accuracy on test set is:  0.691
Epoch   4 Loss:  0.238 Accuracy: 0.9218 Epoch duration: 45.093s
Accuracy on test set is:  0.684
Epoch   5 Loss:  0.215 Accuracy: 0.9314 Epoch duration: 45.089s
Accuracy on test set is:  0.681
model saved at seq_mod/model
Epoch   6 Loss:  0.230 Accuracy: 0.9287 Epoch duration: 45.275s
Accuracy on test set is:  0.684
Epoch   7 Loss:  0.226 Accuracy: 0.9251 Epoch duration: 45.223s
Accuracy on test set is:  0.681
Epoch   8 Loss:  0.227 Accuracy: 0.9266 Epoch duration: 45.256s
Accuracy on test set

Translate on test set

In [0]:
with tf.Session() as sess: 
    restore(sess)
    batch_size = 512
    source_batch, target_batch = next(batch_data(X_test, y_test, batch_size))

    dec_input = np.zeros((len(source_batch), 1)) + len(ltokens)
    for i in range(y_seq_length):
        batch_logits = sess.run(logits,
                    feed_dict = {inputs: source_batch,
                     outputs: dec_input})
        prediction = batch_logits[:,-1].argmax(axis=-1)
        dec_input = np.hstack([dec_input, prediction[:,None]])

    print('Accuracy on test set is: {:>6.3f}'.format(np.mean(dec_input == target_batch)))

INFO:tensorflow:Restoring parameters from seq_mod/model
model restored from seq_mod/model
Accuracy on test set is:  0.687


Let's randomly take two from this test set and see what it spits out:

In [0]:
i = 100
print(dec_input[i,:])
print(ltokens)
seq = ""
for c in dec_input[i,1:]:
    c = int(c)
    if c != 28:
        seq += ltokens[c] 
        
print(seq)

seq = ""
for c in target_batch[i,1:]:
    c = int(c)
    
    if c != 28:
        seq += ltokens[c] 

print("Correct: ", seq)

[33. 28. 27. 24. 27. 25.  7.  6.  7. 26. 25.  1.  5.  0. 26. 21. 23.  4.
 10. 30. 10. 31. 27. 11.  3.  8.]
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '-', '+', '=', '#leq', '#neq', '#geq', '#alpha', '#beta', '#lambda', '#lt', '#gt', 'x', 'y', '^', '#frac', '{', '}', ' ', '#int', 'd', '#sinx', '#cosx', '#tanx']
 #frac {767}{150}x^4-#sinx-#cosx +38
Correct:   #frac {846}{542}x^4-#cosx-#cosx +83
